# Proximal Policy Optimization (PPO)

### Import Libraries


In [ ]:
import random,datetime,gym,os,time,psutil,cv2,scipy.signal
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from gym.spaces import Box, Discrete
from matplotlib import animation
from IPython.display import display, HTML
from collections import deque,namedtuple
%matplotlib inline
gym.logger.set_level(40)
print ("gym version:[%s]"%(gym.__version__))
print ("Pytorch:[%s]"%(torch.__version__))

### Util functions

In [ ]:
def discount_cumsum(x, gamma):
    """
    Compute the discounted cumulative sums of vectors.
    input:
        vector x: [x0,x1,x2,x3]
    output:
        [
         x0 + (gamma)*x1 + (gamma^2)*x2 + (gamma^3)*x3,
         x1 + (gamma)*x2 + (gamma^2)*x3,
         x2 + (gamma)*x3
         x3
        ]
    """
    return scipy.signal.lfilter([1], [1, float(-gamma)], x[::-1], axis=0)[::-1]

def combined_shape(length, shape=None):
    if shape is None:
        return (length,)
    return (length, shape) if np.isscalar(shape) else (length, *shape)

def statistics_scalar(x, with_min_and_max=False):
    """
    Get mean/std and optional min/max of scalar x
    Args:
        x: An array containing samples of the scalar to produce statistics for.
        with_min_and_max (bool): If true, return min and max of x in
            addition to mean and std.
    """
    x = np.array(x, dtype=np.float32)
    global_sum, global_n = np.sum(x), len(x)
    mean = global_sum / global_n
    global_sum_sq = np.sum((x - mean) ** 2)
    std = np.sqrt(global_sum_sq / global_n)  # compute global std
    if with_min_and_max:
        global_min = (np.min(x) if len(x) > 0 else np.inf)
        global_max = (np.max(x) if len(x) > 0 else -np.inf)
        return mean, std, global_min, global_max
    return mean, std

class PPOBuffer:
    """
    A buffer for storing trajectories experienced by a PPO agent interacting
    with the environment, and using Generalized Advantage Estimation (GAE-Lambda)
    for calculating the advantages of state-action pairs.
    """

    def __init__(self, odim, adim, size=5000, gamma=0.99, lam=0.95):
        self.obs_buf = np.zeros(combined_shape(size, odim), dtype=np.float32)
        self.act_buf = np.zeros(combined_shape(size, adim), dtype=np.float32)
        self.adv_buf = np.zeros(size, dtype=np.float32)
        self.rew_buf = np.zeros(size, dtype=np.float32)
        self.ret_buf = np.zeros(size, dtype=np.float32)
        self.val_buf = np.zeros(size, dtype=np.float32)
        self.logp_buf = np.zeros(size, dtype=np.float32)
        self.gamma, self.lam = gamma, lam
        self.ptr, self.path_start_idx, self.max_size = 0, 0, size

    def store(self, obs, act, rew, val, logp):
        """
        Append one timestep of agent-environment interaction to the buffer.
        """
        assert self.ptr < self.max_size  # buffer has to have room so you can store
        self.obs_buf[self.ptr] = obs
        self.act_buf[self.ptr] = act
        self.rew_buf[self.ptr] = rew
        self.val_buf[self.ptr] = val
        self.logp_buf[self.ptr] = logp
        self.ptr += 1

    def finish_path(self, last_val=0):
        """
        Call this at the end of a trajectory, or when one gets cut off
        by an epoch ending. This looks back in the buffer to where the
        trajectory started, and uses rewards and value estimates from
        the whole trajectory to compute advantage estimates with GAE-Lambda,
        as well as compute the rewards-to-go for each state, to use as
        the targets for the value function.

        The "last_val" argument should be 0 if the trajectory ended
        because the agent reached a terminal state (died), and otherwise
        should be V(s_T), the value function estimated for the last state.
        This allows us to bootstrap the reward-to-go calculation to account
        for timesteps beyond the arbitrary episode horizon (or epoch cutoff).
        """
        path_slice = slice(self.path_start_idx, self.ptr)
        rews = np.append(self.rew_buf[path_slice], last_val)
        vals = np.append(self.val_buf[path_slice], last_val)
        # the next two lines implement GAE-Lambda advantage calculation
        deltas = rews[:-1] + self.gamma * vals[1:] - vals[:-1]
        self.adv_buf[path_slice] = discount_cumsum(deltas, self.gamma * self.lam)
        # the next line computes rewards-to-go, to be targets for the value function
        self.ret_buf[path_slice] = discount_cumsum(rews, self.gamma)[:-1]
        self.path_start_idx = self.ptr

    def get(self):
        """
        Call this at the end of an epoch to get all of the data from
        the buffer, with advantages appropriately normalized (shifted to have
        mean zero and std one). Also, resets some pointers in the buffer.
        """
        assert self.ptr == self.max_size  # buffer has to be full before you can get
        self.ptr, self.path_start_idx = 0, 0
        # the next two lines implement the advantage normalization trick
        adv_mean, adv_std = statistics_scalar(self.adv_buf)
        self.adv_buf = (self.adv_buf - adv_mean) / adv_std
        return [self.obs_buf, self.act_buf, self.adv_buf,
                self.ret_buf, self.logp_buf]
    
# Animation function 
def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim.to_jshtml())

def display_frames_as_gif(frames):
    patch = plt.imshow(frames[0])
    plt.axis('off')
    def animate(i):
        patch.set_data(frames[i])
    anim = animation.FuncAnimation(
        plt.gcf(),animate,frames=len(frames),interval=10)
    display(display_animation(anim))
    
print ("Done.")

### Model

In [ ]:
def mlp(odim=24, hdims=[256,256]):
    layers = []
    layers.append(nn.Linear(odim,hdims[0]))
    layers.append(nn.ReLU())
    for idx, hdim in enumerate(hdims):
        if idx < len(hdims)-2:
            layers.append(nn.Linear(hdim,hdims[idx+1]))
            layers.append(nn.ReLU())
        elif idx == len(hdims)-1 :
            layers.append(nn.Linear(hdims[idx-1],hdim))
    return nn.Sequential(*layers)

def gaussian_likelihood(x, mu, log_std):
    EPS = 1e-8
    pre_sum = -0.5 * (((x-mu)/(torch.exp(log_std)+EPS))**2 + 2*log_std + np.log(2*np.pi))
    return torch.sum(pre_sum, axis=1)

class CategoricalPolicy(nn.Module):
    def __init__(self, odim, adim, hdims=[64,64]):
        super(CategoricalPolicy, self).__init__()
        self.net = mlp(odim, hdims=hdims)
        self.logits = nn.Linear(hdims[-1],adim)
    def forward(self, x, a=None):
        output = self.net(x)
        logits = self.logits(output)
        prob = F.softmax(logits)
        dist = torch.distributions.Categorical(probs=prob)
        pi = dist.sample()
        logp_pi = dist.log_prob(pi)
        logp = dist.log_prob(a)
        return pi, logp, logp_pi, pi

class GaussianPolicy(nn.Module):    
    def __init__(self, odim, adim, hdims=[64,64], actv='relu', output_actv=None):
        super(GaussianPolicy, self).__init__()
        self.mu = mlp(odim, hdims=hdims+[adim])
        self.log_std = 0.5 * torch.ones(adim)
    def forward(self, x, a=None):
        mu = self.mu(x)
        log_std = self.log_std
        std = torch.exp(log_std)
        policy = torch.distributions.Normal(mu, std)
        pi = policy.sample()
        logp_pi = gaussian_likelihood(pi, mu, log_std)
        if a is not None:
            logp = gaussian_likelihood(a, mu, log_std)
        else:
            logp = None
        return pi, logp, logp_pi, mu        

class ActorCritic(nn.Module):   # def mlp_actor_critic
    def __init__(self, odim, adim, hdims=[64,64],policy=None, action_space=None):
        super(ActorCritic,self).__init__()
        if (policy is None) and isinstance(action_space, Box):
            # Gaussian policy for continuous actions
            self.policy = GaussianPolicy(odim, adim, hdims)
        elif (policy is None) and isinstance(action_space, Discrete):
            # Categorical policy for discrete actions
            self.policy = CategoricalPolicy(odim, adim, hdims)
        self.vf_mlp = mlp(odim, hdims=hdims+[1])
    def forward(self, x, a=None):
        pi, logp, logp_pi, mu = self.policy(x, a)
        v = self.vf_mlp(x)
        return pi, logp, logp_pi, v, mu
print ("Done.")

### PPO Agent

In [ ]:
def get_envs():
    """
    Get environments
    """
    env = gym.make('Ant-v2',render_mode='rgb_array')
    # Reset env for evaluation
    eval_env = gym.make('Ant-v2',render_mode='rgb_array')
    _,_ = eval_env.reset()
    for _ in range(3): # dummy run for proper rendering
        a = eval_env.action_space.sample()
        o,r,d,_,_ = eval_env.step(a)
        time.sleep(0.01)
    return env,eval_env

class Agent(object):
    def __init__(self,hdims=[256,256],
                 clip_ratio=0.2,target_kl=0.01,
                 gamma=0.99,lam=0.95,
                 pi_lr=3e-4,vf_lr=1e-3,epsilon=1e-2,
                 train_pi_iters=100,train_v_iters=100,
                 epochs=1000,max_ep_len=1000,steps_per_epoch=5000,ep_len_rollout=500,batch_size=4096,
                 print_every=10,evaluate_every=50,update_every=10
                 ):
        
        # Model
        self.hdims           = hdims
        
        # PPO hyperparameters
        self.clip_ratio      = clip_ratio       # clip ratio of PPO
        self.target_kl       = target_kl        # target KL divergence
        
        # PPO Buffer (GAE)
        self.gamma           = gamma            # discount factor
        self.lam             = lam              # GAE lambda
        
        # Update hyperparameters
        self.pi_lr           = pi_lr            # policy learning rate
        self.vf_lr           = vf_lr            # value learning rate
        self.epsilon         = epsilon          # epsilon of adam
        
        # Update
        self.train_pi_iters  = train_pi_iters   # policy update iterations
        self.train_v_iters   = train_v_iters    # value update iterations
        self.epochs          = epochs
        self.max_ep_len      = max_ep_len
        self.steps_per_epoch = steps_per_epoch
        self.ep_len_rollout  = ep_len_rollout
        self.batch_size      = batch_size
        
        self.print_every     = print_every
        self.evaluate_every  = evaluate_every
        self.update_every    = update_every
        
        # Environment
        self.env, self.eval_env = get_envs()
        odim = self.env.observation_space.shape[0]
        adim = self.env.action_space.shape[0]

        # Actor-critic model
        self.actor_critic = ActorCritic(odim,adim,hdims=self.hdims,
                                        action_space=self.env.action_space)
        self.buf = PPOBuffer(odim=odim,adim=adim,
                             size=self.steps_per_epoch, gamma=self.gamma,lam=self.lam)

        # Optimizers
        self.train_pi = optim.Adam(self.actor_critic.policy.parameters(),lr=self.pi_lr)
        self.train_v = optim.Adam(self.actor_critic.vf_mlp.parameters(),lr=self.vf_lr)

    def update_ppo(self, obs, act, adv, ret, logp):
        logp_a_old = logp
        pi_loss, v_loss = 0., 0.

        # Update policy
        for _ in range(self.train_pi_iters):
            _, logp_a, _, _ = self.actor_critic.policy(obs, act)
            ratio = torch.exp(logp_a - logp_a_old)  # pi(a|s) / pi_old(a|s)
            min_adv = torch.where(adv > 0, (1 + self.clip_ratio) * adv, (1 - self.clip_ratio) * adv)
            pi_loss = -torch.mean(torch.minimum(ratio * adv, min_adv))
                
            # Gradient clipping 
            self.train_pi.zero_grad()
            pi_loss.backward()
            self.train_pi.step()

            # KL divergence upper-bound (trust region)
            kl = torch.mean(logp_a_old - logp_a)
            if kl > 1.5 * self.target_kl:
                break

        # Update value
        for _ in range(self.train_v_iters):
            v = torch.squeeze(self.actor_critic.vf_mlp(obs))
            v_loss = F.mse_loss(v, ret)

            self.train_v.zero_grad()
            v_loss.backward()   
            self.train_v.step()
            
        return pi_loss, v_loss

    def train(self):
        start_time = time.time()
        o,_ = self.eval_env.reset()
        r, d, ep_ret, ep_len, n_env_step = 0, False, 0, 0, 0
        for epoch in range(self.epochs):
            o,_ = self.env.reset()
            for t in range(self.steps_per_epoch):
                a, _, logp_t, v_t, _ = self.actor_critic(torch.FloatTensor(o.reshape(1, -1)))
                o2,r,d,_,_ = self.env.step(a.numpy()[0])
                if r < 0.0: r = 0.0
                r = r + 0.01
                ep_ret += r
                ep_len += 1
                n_env_step += 1

                # Save the Experience to our buffer
                self.buf.store(o, a, r, v_t, logp_t)
                o = o2

                terminal = d or (ep_len == self.max_ep_len)
                if terminal or (t == (self.steps_per_epoch - 1)):
                    # if trajectory didn't reach terminal state, bootstrap value target
                    last_val = 0 if d else self.actor_critic.vf_mlp(torch.FloatTensor(o.reshape(1, -1))).detach().numpy()[0][0]
                    self.buf.finish_path(last_val)
                    ep_ret_save = np.copy(ep_ret)
                    o,_ = self.env.reset()
                    ep_ret, ep_len = 0, 0

            # PPO update
            obs, act, adv, ret, logp = [torch.tensor(x) for x in self.buf.get()]
            self.update_ppo(obs, act, adv, ret, logp)
            
            # Print 
            if (epoch == 0) or (((epoch + 1) % self.print_every) == 0):
                print("[%d/%d] ep_ret_save:[%.3f]" % (epoch+1,self.epochs,ep_ret_save))

            # Evaluate 
            if (epoch == 0) or (((epoch + 1) % self.evaluate_every) == 0):
                ram_percent = psutil.virtual_memory().percent  # memory usage
                print("[Eval. start] step:[%d/%d][%.1f%%] #step:[%.1e] time:[%s] ram:[%.1f%%]." %
                      (epoch + 1, self.epochs, epoch / self.epochs * 100,
                       n_env_step,
                       time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time)),
                       ram_percent)
                      )
                o,_ = self.eval_env.reset()
                d, ep_ret, ep_len = False, 0, 0
                frames = []
                while not (d or (ep_len == self.max_ep_len)):
                    a, _, _, _ = self.actor_critic.policy(torch.FloatTensor(o.reshape(1, -1)))
                    o,r,d,_,_ = self.eval_env.step(a.numpy()[0])
                    if r < 0.0: r = 0.0
                    r = r + 0.01
                    frame = self.eval_env.render()
                    texted_frame = cv2.putText(
                        img=np.copy(frame),
                        text='tick:[%d]'%(ep_len),
                        org=(80,30),fontFace=2,fontScale=0.8,color=(0,0,255),thickness=1)
                    if (ep_len%5) == 0:
                        frames.append(texted_frame)
                    ep_ret += r  # compute return
                    ep_len += 1
                display_frames_as_gif(frames)
                print("[Eval. done] episode return:[%.4f] length:[%d]" % (ep_ret, ep_len))
                
print ("Done.")

### Train an Ant agent with PPO

In [ ]:
A = Agent(hdims=[256,256],
          clip_ratio=0.5,target_kl=0.01,
          gamma=0.99,lam=0.95,
          pi_lr=1e-4,vf_lr=1e-4,epsilon=1e-2,
          train_pi_iters=100,train_v_iters=100,
          epochs=10000,max_ep_len=500,steps_per_epoch=5000,ep_len_rollout=500,batch_size=4096,
          print_every=20,evaluate_every=50,update_every=10)
A.train()